# imports

In [1]:
import pandas as pd  
import numpy as np 
import re
import os 

In [2]:
# webdriver: provides the API for accessing and controlling a web browser.
from selenium import webdriver

# Keys: provides keys that can be used for keyboard interactions.
from selenium.webdriver.common.keys import Keys

# By: provides mechanisms to locate elements on a web page.
from selenium.webdriver.common.by import By

# WebDriverWait: provides a way to wait for a specific condition to occur before proceeding with the code execution.
from selenium.webdriver.support.ui import WebDriverWait

# expected_conditions: provides a set of predefined conditions that can be used with WebDriverWait to wait for an element to become available, clickable, or visible.
from selenium.webdriver.support import expected_conditions as EC

In [3]:
from bs4 import BeautifulSoup

# Acquire Selenium

In [4]:
season_list = [1,2,2018,2019,2020,2021]

# Season 1 / 2015

In [5]:
# Create an instance of the Chrome browser
driver = webdriver.Chrome()

# Navigate to the BattleBots website
driver.get("https://battlebots.com/season-1-robots/")

# Get the page source
html = driver.page_source

# Parse the HTML using BeautifulSoup
soup = BeautifulSoup(html, "html.parser")


In [6]:
# Find all the <div> elements with class="cbp-wrapper"
div_elements = soup.find_all("div", {"class": "cbp-wrapper"})



In [7]:
links = []
# Find all the <div> elements with class="cbp-wrapper"
div_elements = soup.find_all("div", {"class": "cbp-wrapper"})

# Iterate over the list of matching <div> elements
for div in div_elements:
    # Find all the <div> elements with class="more-button has-love" within the current <div> element
    more_buttons = div.find_all("div", {"class": "more-button has-love"})
    
    # For example, you could extract the href attribute of the <a> element within each <div>
    for button in more_buttons:
        link = button.find("a")["href"]
        links.append(link)


# Test One Link
    

In [8]:
test = links[0]

In [9]:
tab = pd.read_html('https://battlebots.com/robot/beta/')

In [10]:
df = tab[0]


In [11]:
df = df.drop(columns = ['Season 2', 'Season 1'])

In [12]:
df = df.melt(id_vars=["Stats"], var_name="Season", value_name="Value").T

In [13]:
df = df.set_axis(df.iloc[0], axis=1)

In [14]:
df

Stats,Total matches,Win percentage,Total wins,Losses,Knockouts,KO percentage,Average knockout time,Knockouts against,KO against percentage,Judges decision wins
Stats,Total matches,Win percentage,Total wins,Losses,Knockouts,KO percentage,Average knockout time,Knockouts against,KO against percentage,Judges decision wins
Season,Career,Career,Career,Career,Career,Career,Career,Career,Career,Career
Value,4,75%,3,1,1,25%,180s,0,0%,2


In [15]:
# drop the first row (which is now the header)
df = df.drop(df.index[[0,1]])
df = df.reset_index()


In [19]:
df['robot'] = 'Beta'

In [20]:
df = df[['robot', 'Total matches', 'Win percentage', 'Total wins', 'Losses',
       'Knockouts', 'KO percentage', 'Average knockout time',
       'Knockouts against', 'KO against percentage', 'Judges decision wins']]

In [21]:
df

Stats,robot,Total matches,Win percentage,Total wins,Losses,Knockouts,KO percentage,Average knockout time,Knockouts against,KO against percentage,Judges decision wins
0,Beta,4,75%,3,1,1,25%,180s,0,0%,2


In [30]:
links

['https://battlebots.com/robot/beta/',
 'https://battlebots.com/robot/bite-force/',
 'https://battlebots.com/robot/bronco/',
 'https://battlebots.com/robot/bull-dog/',
 'https://battlebots.com/robot/captain-shrederator/',
 'https://battlebots.com/robot/chomp/',
 'https://battlebots.com/robot/chronic/',
 'https://battlebots.com/robot/complete-control/',
 'https://battlebots.com/robot/counter-revolution/',
 'https://battlebots.com/robot/ghost-raptor/',
 'https://battlebots.com/robot/hypershock/',
 'https://battlebots.com/robot/icewave/',
 'https://battlebots.com/robot/lock-jaw/',
 'https://battlebots.com/robot/mohawk/',
 'https://battlebots.com/robot/nightmare/',
 'https://battlebots.com/robot/overdrive/',
 'https://battlebots.com/robot/overhaul/',
 'https://battlebots.com/robot/plan-x/',
 'https://battlebots.com/robot/radioactive/',
 'https://battlebots.com/robot/razorback/',
 'https://battlebots.com/robot/splatter/',
 'https://battlebots.com/robot/stinger/',
 'https://battlebots.com/ro

In [61]:
all_tables = pd.DataFrame()  # initialize an empty dataframe to hold all tables

for link in links:
    try:
        tables = pd.read_html(link)
    except ValueError:
        # No tables found, skip to the next link
        continue
        
    # check if any tables were found
    if len(tables) > 0:
        # assigning first table to variable                   
        table = tables[0]
        
        # dropping season 1 and 2, can't fit it appropriately  
        try: 
            table = table.drop(columns=['Season 2', 'Season 1'])
            table = table.reset_index(drop=True)
        except KeyError:
            pass
                 
        # meltting a dataframe
        table = table.melt(id_vars=["Stats"], var_name="Season", value_name="Value").T
                  
        # set columns names from the first row                       
        table = table.set_axis(table.iloc[0], axis=1)
                         
        # drop the first row (which is now the header)
        table = table.drop(table.index[[0,1]])
    
        url = f'{link}'
        match = re.search(r'/([^/]+)/?$', url)
        if match:
            table['robot'] = match.group(1)
            # add the table to the collective dataframe
            all_tables = all_tables.append(table)
    else:
        print(f"No tables found for {link}. Skipping...")



/var/folders/d0/5dr5q0n145vcn3_m9dxy1xq80000gn/T/ipykernel_46410/1047331585.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_tables = all_tables.append(table)
/var/folders/d0/5dr5q0n145vcn3_m9dxy1xq80000gn/T/ipykernel_46410/1047331585.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_tables = all_tables.append(table)
/var/folders/d0/5dr5q0n145vcn3_m9dxy1xq80000gn/T/ipykernel_46410/1047331585.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_tables = all_tables.append(table)
/var/folders/d0/5dr5q0n145vcn3_m9dxy1xq80000gn/T/ipykernel_46410/1047331585.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_tables = a

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [62]:
all_tables

Stats,Total matches,Win percentage,Total wins,Losses,Knockouts,KO percentage,Average knockout time,Knockouts against,KO against percentage,Judges decision wins,robot
Value,4,75%,3,1,1,25%,180s,0,0%,2,beta
Value,8,88%,7,1,3,38%,114 s,0,0%,4,bite-force
Value,8,75%,6,2,6,75%,70 s,2,25%,0,bronco
Value,3,33%,1,2,1,33%,124 s,2,67%,0,captain-shrederator
Value,6,50%,3,3,1,17%,128 s,3,50%,2,chomp
Value,3,33%,1,2,0,0%,NaN,33%,0%,1,complete-control
Value,1,0%,0,1,0,0%,NaN,1,100%,0,counter-revolution
Value,6,50%,3,3,2,33%,113 s,3,50%,1,ghost-raptor
Value,5,60%,3,2,3,60%,86 s,2,40%,0,hypershock
Value,5,60%,3,2,2,40%,81 s,1,40%,0,icewave
